In [9]:
DEBEZIUM_URL = 'http://localhost:8083/connectors'

pg_connector_config = {
  "name": "pg-connector",
  "config": {
    "connector.class": "io.debezium.connector.postgresql.PostgresConnector",
    "plugin.name": "pgoutput",
    
    "database.hostname": "postgres_master",
    "database.port": "5432",
    "database.user": "postgres",
    "database.password": "postgres",
    "database.dbname" : "postgres",
    "database.server.name": "pg-dev",
    
    "table.include.list": "public.(.*)",
    "heartbeat.interval.ms": "5000",
    "slot.name": "dbname_debezium",
    "publication.name": "dbname_publication",
      
    "topic.creation.default.cleanup.policy": "delete",
    "topic.creation.default.partitions": "1",
    "topic.creation.default.replication.factor": "1",
    "topic.creation.default.retention.ms": "604800000",
    "topic.creation.enable": "true",
    "topic.prefix": "postgres"
  }
}

sink_connectors_info = {
    'postgres.public.categories': 'category_id',
    'postgres.public.customers': 'customer_id',
    'postgres.public.manufacturers': 'manufacturer_id',
    'postgres.public.deliveries': 'delivery_id',
    'postgres.public.price_change': 'product_id,price_change_ts',
    'postgres.public.products': 'product_id',
    'postgres.public.purchase_items': 'product_id,purchase_id',
    'postgres.public.purchases': 'purchase_id',
    'postgres.public.stores': 'store_id'
}

In [11]:
import requests
from typing import Dict, Any


def add_connector(connector_dict: Dict[str, Any]) -> None:
    response = requests.post(
        DEBEZIUM_URL, 
        json=connector_dict
    )
    assert response.status_code < 400, response.text


def add_pg_connector() -> None:
    add_connector(pg_connector_config)


def add_sink_connectors() -> None:
    for topic, record_keys in sink_connectors_info.items():
        add_connector(
            {
            "name": topic.replace('.', '-'),
            "config": {
                "connector.class": "io.debezium.connector.jdbc.JdbcSinkConnector",
                "topics": topic,
                "connection.url": "jdbc:postgresql://postgres_dwh:5432/postgres",
                "connection.username": "postgres",
                "connection.password": "postgres",
                "tasks.max":"1",
                "insert.mode": "upsert",
                "delete.enabled": "false",
                "primary.key.mode": "record_key",
                "primary.key.fields": record_keys,
                "schema.evolution": "basic"
                }
            }
        )

In [13]:
add_pg_connector()

In [15]:
add_sink_connectors()